In [1]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [2]:
import subprocess
from pathlib import Path
from collections import Counter

In [5]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

if True:
    cmd = [
        'python3', 'main.py',
        #'--api-base-url', 'http://localhost:8000/v1',
        '--api-key', 'xxx',
        '--model-name', 'unsloth/gemma-3-4b-it',
        '--chat-template-model-id', 'unsloth/gemma-3-4b-it',
        '--logging-level', 'INFO',
        '--slop-phrases-file', 'banlists/slop_phrases.json',
        '--top-n-slop-phrases', '500',
        '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
        '--max-new-tokens', '500',
        '--prompt', 'Write a story about a princess. '
    ]
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

Running: python3 main.py --api-key xxx --model-name unsloth/gemma-3-4b-it --chat-template-model-id unsloth/gemma-3-4b-it --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 500 --prompt Write a story about a princess. 
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
--- Generation start ---

--- Generated Output (stream) ---


2025-05-18 17:11:19,326 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates survived even after relaxation.


Princess Aurelia of Silverwood wasn't known for her embroidery or her dancing. She wasn't known for her grace or her polite smiles. Aurelia was known for her mud. She was, quite simply, obsessed with mud. Not the refined, sculpted mud of a royal garden, but the glorious, glorious, dripping, squelching mud of the Silverwood Forest.

While other princesses her age practiced courtly manners and learned the names of rare flowers, Aurelia spent her days exploring the deepest, darkest corners of the forest, her boots caked in varying shades of ochre, grey, and deep, earthy black. The King, a weary man burdened by the weight of his crown and the expectation of a suitable bride, tolerated her eccentricities with a resigned shrug. Her mother, the Queen, simply worried.

“Aurelia, darling,” the Queen would say, smoothing a silk gown, “you’ll ruin your shoes. And your reputation!”

But Aurelia wasn’t interested in reputation. She was interested in the way mud felt between her toes, the way it sme

In [7]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

# Make sure you've launched the vllm openai-compatible server first. You can use something like:

# vllm serve unsloth/gemma-3-4b-it --port 8000 --max-model-len 2500 --served-model-name unsloth/gemma-3-4b-it --gpu-memory-utilization 0.95 --dtype bfloat16 --api-key xxx


THREADS  = 40
MAX_PROMPTS = 40

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8000/v1',
    '--api-key', 'xxx',
    '--model-name', 'unsloth/gemma-3-4b-it',
    '--output-jsonl', OUT_JSONL,
    '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'INFO',
    '--slop-phrases-file', 'banlists/slop_phrases.json',
    '--top-n-slop-phrases', '500',
    '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '2000',
    '--request-mode', 'chunk',  # chunk / stream (stream mode might be buggy)
    '--chunk-size', '20', # how many tokens we request from the api before checking it for new violations
    '--chat-template-model-id', "unsloth/gemma-3-4b-it",
    '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python3 main.py --api-key xxx --model-name unsloth/gemma-3-4b-it --output-jsonl creative_writing_generations.jsonl --input-hf-dataset Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT --hf-dataset-split train --threads 40 --max-prompts 40 --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 2000 --request-mode chunk --chunk-size 20 --chat-template-model-id unsloth/gemma-3-4b-it --tdpo-pairs-jsonl tdpo-pairs.jsonl
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
Extracting HF prompts:   0%|          | 39/177477 [00:00<00:05, 34781.60prompt/s]
Preparing to process 40 new prompts in this run.
BANNED (prompt_idx=12, type=slop_phrase, phrase='delicate') 
BANNED (prompt_idx=23, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:[^\s]+\s*){1,6}?[,;:—–-]?\s*but\s+(?:also\s+)?', match='Not a sharp, biting regret, but ')

2025-05-18 17:15:33,616 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates survived even after relaxation.


BANNED (prompt_idx=21, type=slop_phrase, phrase='unsettling')            
BANNED (prompt_idx=18, type=slop_phrase, phrase='overwhelming')          
BANNED (prompt_idx=4, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:[^\s]+\s*){1,6}?[,;:—–-]?\s*but\s+(?:also\s+)?', match='Not a dramatic, weeping regret, mind you, but ')
BANNED (prompt_idx=2, type=slop_phrase, phrase='etched')                 
BANNED (prompt_idx=23, type=slop_phrase, phrase='clung')                 
BANNED (prompt_idx=15, type=slop_phrase, phrase='polished')              
BANNED (prompt_idx=16, type=slop_phrase, phrase='swirling')              
BANNED (prompt_idx=31, type=slop_phrase, phrase='unsettling')            
BANNED (prompt_idx=25, type=slop_phrase, phrase='rhythm')                
BANNED (prompt_idx=10, type=slop_phrase, phrase='felt like an eternity') 
BANNED (prompt_idx=29, type=slop_phrase, phrase='etched')                
BANNED (prompt_idx=6, type=ngram, ngram='almost imperceptible')          
BANNE

2025-05-18 17:15:34,321 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates survived even after relaxation.


BANNED (prompt_idx=21, type=slop_phrase, phrase='hollow')                
BANNED (prompt_idx=32, type=slop_phrase, phrase='profound')              
BANNED (prompt_idx=15, type=slop_phrase, phrase='polished')              
BANNED (prompt_idx=7, type=slop_phrase, phrase='scent')                  
BANNED (prompt_idx=31, type=slop_phrase, phrase='air hung thick')        
BANNED (prompt_idx=16, type=slop_phrase, phrase='obsidian')              
BANNED (prompt_idx=2, type=ngram, ngram='carefully constructed')         
BANNED (prompt_idx=6, type=slop_phrase, phrase='echo')                   
BANNED (prompt_idx=29, type=slop_phrase, phrase='etched')                
BANNED (prompt_idx=28, type=slop_phrase, phrase='anticipation')          
BANNED (prompt_idx=8, type=slop_phrase, phrase='rhythm')                 
BANNED (prompt_idx=17, type=slop_phrase, phrase='frantic')               
BANNED (prompt_idx=19, type=slop_phrase, phrase='stark')                 
BANNED (prompt_idx=22, type=slop_phras

...

2025-05-18 17:22:58,184 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates survived even after relaxation.


BANNED (prompt_idx=31, type=slop_phrase, phrase='shimmering')                     
Batch Generating:  98%|█████████▊| 39/40 [07:28<00:02,  2.28s/prompt, 5.1 tok/s]  

2025-05-18 17:22:58,834 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates survived even after relaxation.


BANNED (prompt_idx=31, type=slop_phrase, phrase='shimmering')                   
BANNED (prompt_idx=31, type=slop_phrase, phrase='utterly')                      
BANNED (prompt_idx=31, type=slop_phrase, phrase='rain continued to fall')       
BANNED (prompt_idx=31, type=slop_phrase, phrase='profound')                      
Batch Generating: 100%|██████████| 40/40 [07:32<00:00, 11.32s/prompt, 13.8 tok/s]
Finished processing 40 prompts in this run in 452.94s.
Overall average throughput for this run: 144.51 tok/s.
Results appended to creative_writing_generations.jsonl


CompletedProcess(args=['python3', 'main.py', '--api-key', 'xxx', '--model-name', 'unsloth/gemma-3-4b-it', '--output-jsonl', 'creative_writing_generations.jsonl', '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT', '--hf-dataset-split', 'train', '--threads', '40', '--max-prompts', '40', '--logging-level', 'INFO', '--slop-phrases-file', 'banlists/slop_phrases.json', '--top-n-slop-phrases', '500', '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json', '--max-new-tokens', '2000', '--request-mode', 'chunk', '--chunk-size', '20', '--chat-template-model-id', 'unsloth/gemma-3-4b-it', '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'], returncode=0)